In [1]:
#importações:
from pyspark.sql.types import StructType

In [2]:
!hdfs dfs -ls /user/silas/data/iris
#contagem de linha dos arquivos
#bezdekiris.data 150 linhas
#iris.data 150 linhas

Found 3 items
-rw-r--r--   3 root supergroup       4551 2021-10-19 23:40 /user/silas/data/iris/bezdekIris.data
-rw-r--r--   3 root supergroup       4551 2021-10-19 23:40 /user/silas/data/iris/iris.data
-rw-r--r--   3 root supergroup       2998 2021-10-19 23:40 /user/silas/data/iris/iris.names


In [3]:
#especificando um schema *obrigatório
iris_schema = StructType().add("sepal_lenght","float").add("sepal_width","float").add("petal_lenght","float").add("petal_width","float").add("class","string")
print(iris_schema)

StructType(List(StructField(sepal_lenght,FloatType,true),StructField(sepal_width,FloatType,true),StructField(petal_lenght,FloatType,true),StructField(petal_width,FloatType,true),StructField(class,StringType,true)))


In [4]:
iris = spark.read.schema(iris_schema).csv("/user/silas/data/iris/*.data")
iris.show(5)
iris.count()

+------------+-----------+------------+-----------+-----------+
|sepal_lenght|sepal_width|petal_lenght|petal_width|      class|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|
+------------+-----------+------------+-----------+-----------+
only showing top 5 rows



300

In [ ]:
iris_bez = spark.read.csv("/user/silas/data/iris/bezdekIris.data")
iris_bez.count()

In [ ]:
iris_dat = spark.read.csv("/user/silas/data/iris/iris.data")
iris_dat.count()

In [5]:
#criando o stream
iris_stream = spark.readStream.schema(iris_schema).csv("/user/silas/data/iris/*.data")

In [ ]:
iris_stream.printSchema()

In [6]:
#diretório para salvar 
iris_saida = iris_stream.writeStream.format("csv")\
.option("checkpointLocation","/user/silas/stream_iris/check")\
.option("path","/user/silas/stream_iris/path").start()

In [ ]:
iris_saida.id
iris_saida.lastProgress
iris_saida.status


In [7]:
#verificação do checkpoint
!hdfs dfs -ls /user/silas/stream_iris/check

Found 4 items
drwxr-xr-x   - root supergroup          0 2021-10-30 23:44 /user/silas/stream_iris/check/commits
-rw-r--r--   3 root supergroup         45 2021-10-30 23:37 /user/silas/stream_iris/check/metadata
drwxr-xr-x   - root supergroup          0 2021-10-30 23:44 /user/silas/stream_iris/check/offsets
drwxr-xr-x   - root supergroup          0 2021-10-30 23:37 /user/silas/stream_iris/check/sources


In [8]:
!hdfs dfs -ls /user/silas/stream_iris/path

Found 3 items
drwxr-xr-x   - root supergroup          0 2021-10-30 23:44 /user/silas/stream_iris/path/_spark_metadata
-rw-r--r--   2 root supergroup       4550 2021-10-30 23:44 /user/silas/stream_iris/path/part-00000-062b0a2c-2cc2-4248-b9cf-823b670c6496-c000.csv
-rw-r--r--   2 root supergroup       4550 2021-10-30 23:44 /user/silas/stream_iris/path/part-00001-dd04f8bc-f54e-4fb0-965e-ad09c0805e1f-c000.csv


In [9]:
spark.read.csv("/user/silas/stream_iris/path/part-00000-062b0a2c-2cc2-4248-b9cf-823b670c6496-c000.csv").count()

150